# House price

---

Le but de ce notebook est d'étudier un jeu de données de maison pour ainsi deviner le prix des maisons à partir de leur descriptif. Le jeu de données est accompagné d'un descriptif des variables. Si jamais vous n'arrivez pas à le récupérer sur git, il est aussi disponible à [cette adresse](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data).

In [ ]:
# Import des bibliothèques pertinentes

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from xgboost import XGBRegressor

# Partie 1 : analyse de la qualité des données

1) Importer les données. Combien de lignes possède-t-on ? Combien de colonnes ?
2) Bon sang ! ça fait beaucoup de colonnes ! Parmi les colonnes, combien possèdent moins de 10 % de données non nulles ? Ce chiffre vous étonne-t-il ?
2) Quelles sont les 10 colonnes avec le plus de variables manquantes ?
3) Parmi les colonnes, combien possèdent 90 % de valeurs identiques ? Ce chiffre vous étonne-t-il ?
4) Y a-t-il des doublons dans le jeu de données ?

In [ ]:
# Récupération des données
train_house = pd.read_csv("house_data/train.csv")

In [ ]:
# Question 2)
print(f"Notre dataframe contient {len(train_house)} lignes et {len(train_house.columns)} colonnes.")

In [ ]:
train_house.head()

In [ ]:
train_house.info()

In [ ]:
train_house.describe()

In [ ]:
# Question 2)
print(f"Il y a {sum(train_house.isna().sum() > (len(train_house) * 0.9))} colonnes avec plus de 90 % de données vides.")

In [ ]:
# Afin de répondre à la question, on essaye d'identifier les trois colonnes avec des données manquantes.
train_house.isna().sum()[train_house.isna().sum() > (len(train_house) * 0.9)]
# On se rend compte qu'il s'agit de colonnes en rapport avec des détails "de luxe" (piscines et Allée).
# La plupart des maisons ne doivent pas en avoir.

In [ ]:
train_house.groupby(["Street"]).count()["Id"].max()

In [ ]:
# Question 3) On possède un petit dataframe. On peut faire un groupby et compter le nombre de valeurs égales.
# Note : pour les plus grosses bases de données, on cherchera des méthodes plus intelligentes (regarder la médiane et les quartiles)
list_col_identical = []
for my_col in train_house.columns:
    if my_col != "Id":
        if train_house.groupby([my_col]).count()["Id"].max() >= (len(train_house) * 0.9):
            list_col_identical.append(my_col)

In [ ]:
print(f"On possède {len(list_col_identical)} colonnes avec 90 % des valeurs identiques au moins. Ces colonnes sont les suivantes : {list_col_identical}.")

# Partie 2) Compréhension du prix des maisons.
1) Tracez la distribution du prix des maisons. Que remarque-t-on ?
2) De façon intuitive, le prix dépend de la taille. Tracez le prix des maisons en fonction de leur taille. Y a-t-il une relation linéaire ? Si oui, tracez sur votre graphe la droite.
3) Identifiez les outliers de la question d'avant.
4) Quelles sont les variables les plus corrélées avec le prix des maisons ?
5) Certaines variables semblent redondantes. Quelles sont les variables avec une très forte corrélation entre elles ?
5) Trouvez une visualisation pertinente qui relie le lien entre le prix des maisons et leur note globale.
6) Les maisons récentes sont-elles plus chères que les maisons anciennes ?

Récapitulatif de ce que l'on sait.

Relation linéaire entre taille de la maison et son prix. Great. Relation avec la qualité de la maison. Pas de relation préciser avec année de vente. Relation avec année de construction. Pas mal de colonnes vides et inutiles + redondances

# Partie 3) Préparons la partie machine learning
1) Au vue de la partie 1, quelles colonnes vous semblent pertinentes à ôter de notre dataframe ?
2) Que faire des outliers identifiés en partie 2 ?
2) Gérez les colonnes avec des valeurs manquantes.
2) Y a-t-il des variables qui mériteraient d'être transformées ? (par exemple, on possède 4 variables en rapport avec les salles de bain)
2) On constate que certaines de nos données catégoriques possèdent une hierarchie. Catégorisez-les de façon logique.
3) Catégorisez le reste des données puis normalisez les. (Note : supprimez les catégories trop rares).
3) Faites une régression linéaire. Visualisez les coefficients pour les différentes colonnes. Quelles sont les colonnes avec les plus forts coefficients ? Cela vous semble-t-il logique ?
3) Faites un ridge regression avec différentes valeurs de Alpha. Visualisez les coefficients pour les différentes colonnes. Quelles sont les colonnes avec les plus forts coefficients ? Cela vous semble-t-il logique ?
3) Essayez de faire une prédiction avec un arbre de décision (DecisionTreeRegressor). Ce modèle vous emble-t-il adapté à notre problème ?
3) Il est temps de dévoiler notre puissance ! Utilisez un XGBoost et plions ce problème comme la crêpe insignifiante qu'il a toujours été !